In [1]:
import numpy as np
import pandas as pd

## Hierarchical indexing

Representing high dimensions in a low dimensional form.

### Series

In [2]:
ser = pd.Series(np.random.standard_normal(12),
                index=[['a'] * 4 + ['b'] * 2 + ['c'] * 6,
                       [1, 2, 3, 4, 1, 2, 1, 3, 5, 7, 9, 11]])
ser

a  1     0.724557
   2     1.060909
   3     0.942034
   4     0.557756
b  1    -0.348887
   2     0.721037
c  1    -1.012563
   3     0.963239
   5    -0.528505
   7     0.211334
   9    -1.343941
   11    0.353050
dtype: float64

Internal representation of index:

In [3]:
ser.index

MultiIndex([('a',  1),
            ('a',  2),
            ('a',  3),
            ('a',  4),
            ('b',  1),
            ('b',  2),
            ('c',  1),
            ('c',  3),
            ('c',  5),
            ('c',  7),
            ('c',  9),
            ('c', 11)],
           )

In [4]:
ser.loc['b']

1   -0.348887
2    0.721037
dtype: float64

In [5]:
ser['a':'b']

a  1    0.724557
   2    1.060909
   3    0.942034
   4    0.557756
b  1   -0.348887
   2    0.721037
dtype: float64

In [6]:
ser[:, 3]

a    0.942034
c    0.963239
dtype: float64

Turning into a dataframe by unstacking, which can also be stacked.

In [7]:
ser.unstack()

,1,2,3,4,5,7,9,11
a,0.724557,1.060909,0.942034,0.557756,NaN,NaN,NaN,NaN
b,-0.348887,0.721037,NaN,NaN,NaN,NaN,NaN,NaN
c,-1.012563,NaN,0.963239,NaN,-0.528505,0.211334,-1.343941,0.35305


In [8]:
ser.unstack().stack()

a  1     0.724557
   2     1.060909
   3     0.942034
   4     0.557756
b  1    -0.348887
   2     0.721037
c  1    -1.012563
   3     0.963239
   5    -0.528505
   7     0.211334
   9    -1.343941
   11    0.353050
dtype: float64

### DataFrame

In [9]:
df = pd.DataFrame(np.arange(16).reshape((4,4)),
                  index=[['a', 'a', 'b', 'b'], [0, 1, 1, 3]],
                  columns=[['Bacon', 'Bacon', 'Eggs', 'Eggs'], ['Piggy', 'Puppy', 'Piggy', 'Kitty']])
df.index.names = ['1st', '2nd']
df.columns.names = ['Food', 'Customer']
df

Food     Bacon        Eggs      
Customer Piggy Puppy Piggy Kitty
1st 2nd                         
a   0        0     1     2     3
    1        4     5     6     7
b   1        8     9    10    11
    3       12    13    14    15

In [10]:
df.columns.nlevels, df.index.nlevels

(2, 2)

In [11]:
df['Bacon']

Customer  Piggy  Puppy
1st 2nd               
a   0         0      1
    1         4      5
b   1         8      9
    3        12     13

In [12]:
df.loc['a']

Food     Bacon        Eggs      
Customer Piggy Puppy Piggy Kitty
2nd                             
0            0     1     2     3
1            4     5     6     7

This is another way of creating a multi index.

In [13]:
pd.MultiIndex.from_arrays([['Bacon', 'Bacon', 'Eggs', 'Eggs'],
                          ['Piggy', 'Puppy', 'Piggy', 'Kitty']],
                          names=['Food', 'Customer'])

MultiIndex([('Bacon', 'Piggy'),
            ('Bacon', 'Puppy'),
            ( 'Eggs', 'Piggy'),
            ( 'Eggs', 'Kitty')],
           names=['Food', 'Customer'])

In [14]:
df.columns

MultiIndex([('Bacon', 'Piggy'),
            ('Bacon', 'Puppy'),
            ( 'Eggs', 'Piggy'),
            ( 'Eggs', 'Kitty')],
           names=['Food', 'Customer'])

### Reordering and Sorting levels

`df.swaplevel` can swap either columns or rows.

In [15]:
df

Food     Bacon        Eggs      
Customer Piggy Puppy Piggy Kitty
1st 2nd                         
a   0        0     1     2     3
    1        4     5     6     7
b   1        8     9    10    11
    3       12    13    14    15

In [16]:
df.swaplevel()

Food     Bacon        Eggs      
Customer Piggy Puppy Piggy Kitty
2nd 1st                         
0   a        0     1     2     3
1   a        4     5     6     7
    b        8     9    10    11
3   b       12    13    14    15

`df.sort_index` sorts by labels along axis.

In [17]:
df.sort_index(ascending=False, axis=1)

Food      Eggs       Bacon      
Customer Piggy Kitty Puppy Piggy
1st 2nd                         
a   0        2     3     1     0
    1        6     7     5     4
b   1       10    11     9     8
    3       14    15    13    12

In [18]:
df.sort_index(level=1, ascending=False, axis=1)

Food     Bacon  Eggs Bacon  Eggs
Customer Puppy Piggy Piggy Kitty
1st 2nd                         
a   0        1     2     0     3
    1        5     6     4     7
b   1        9    10     8    11
    3       13    14    12    15

Selection performance is better on `sort_index()` dataframes.

### Index to Column to Index 

Column can become an index with `set_index`.

In [19]:
df2 = df['Bacon']
df2

Customer  Piggy  Puppy
1st 2nd               
a   0         0      1
    1         4      5
b   1         8      9
    3        12     13

In [20]:
df2.set_index(['Piggy'])

Customer,Puppy
Piggy,
0,1
4,5
8,9
12,13


In [21]:
df2.set_index(['Piggy'], drop=False)

Customer,Piggy,Puppy
Piggy,,
0,0,1
4,4,5
8,8,9
12,12,13


Or remove all the index by `reset_index`

In [22]:
df2.reset_index()

Customer,1st,2nd,Piggy,Puppy
0,a,0,0,1
1,a,1,4,5
2,b,1,8,9
3,b,3,12,13


## Combining and Merging datasets

### Database-Style DataFrame Joins

`pandas.merge` is a database-style join. It can perform one-to-one, one-to-many, and many-to-many operations.

There are couple different ways to join them using `how`:
  - `left`: SQL left outer join.
  - `right`: SQL right outer join.
  - `outer`: SQL full outer join. Keys sorted lexicographically.
  - `inner`: SQL inner join.
  - `cross`: cartesian product from both frames. Left keys' orders are saved.

In [23]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df2

,key,data2
0,a,0
1,b,1
2,d,2


By default, it performs inner join, but other options are also available. Although specifying key is not required, it is a good to specify.

In [25]:
pd.merge(df1, df2, on='key', how='inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [26]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


In [27]:
pd.merge(df1, df2, on='key', how='right')

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,d,<NA>,2


In [28]:
pd.merge(df1, df2, on='key', how='outer')

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


For each dataframe, column names might be different. Instead of changing names to match, we can specify the keys from each dataframe to match.

In [29]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [30]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [31]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


We can join on multiple keys from each dataframe.

In [32]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [33]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [34]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


Key names can also get suffixes. This is useful when column names overlap. If no suffix is given, it adds automatically.

In [35]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

Some data may have index that could be used as keys for merging. We can set `right_index` or `left_index` to `True`.

In [36]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [37]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
right1

,group_val
a,3.5
b,7.0


In [38]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


Hierarchically indexed data can also be merged easily.

In [39]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [40]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth_index

MultiIndex([('Nevada', 2001),
            ('Nevada', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2001),
            (  'Ohio', 2002)],
           )

In [41]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [42]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


### Join

`pd.DataFrame.join` is like a simpler version of `pd.merge`. It automatically left joins on index.

In [43]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [44]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [45]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


Same thing with `pd.merge`:

In [46]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


We can join multiple dataframes with a list of dataframes.

In [47]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [48]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [49]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [50]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### Concatenating Along an Axis

We can also concatenate or stack along axis. Here is how it is done in numpy.

In [51]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [52]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [53]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

Here is how you concatenate in pandas. The only difference is that pandas objects have index.

In [54]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [55]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [56]:
pd.concat([s1,s2,s3], axis=1)

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


Concatenating series with some overlapping index.

In [57]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [58]:
pd.concat([s1, s4], axis=1)

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [59]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


Concatenating with an identifiable result. `keys` are specified to create hierarchical index.

In [60]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [61]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


Concatenating along columns creates a dataframe.

In [62]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [63]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [64]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df2

,three,four
a,5,6
c,7,8


In [65]:
pd.concat([df1, df2], axis="columns")

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [66]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [67]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [68]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

Concatenating ignoring index.

In [69]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df1

,a,b,c,d
0,0.016571,0.143160,0.059655,0.440706
1,-0.265080,-0.339631,-0.614727,-0.257513
2,-0.694321,-0.815525,-0.726871,-0.685082


In [70]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df2

,b,d,a
0,-0.049512,0.879904,0.253678
1,0.106942,0.463415,-0.235605


In [71]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.016571,0.143160,0.059655,0.440706
1,-0.265080,-0.339631,-0.614727,-0.257513
2,-0.694321,-0.815525,-0.726871,-0.685082
3,0.253678,-0.049512,NaN,0.879904
4,-0.235605,0.106942,NaN,0.463415


### Combining data with overlap

We can combine data with some overlapping index. Result has the union of index. First, this is how to combine without index, using numpy.

In [72]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [73]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [74]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Another way to combine is considering index. Index is ordered for the result.

In [75]:
a.sort_index()

a    NaN
b    4.5
c    3.5
d    0.0
e    2.5
f    NaN
dtype: float64

In [76]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [77]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

Same principle on datafrmaes: union of columns and index.

In [78]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [79]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [80]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting

### Reshaping with Hierarchical indexing

In [81]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


`stack` stacks columns to rows.

In [82]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

`unstack` unstacks innermost rows into columns. But the level can be specified.

In [83]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [84]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [85]:
result.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


`stack` can also specify levels.

In [86]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
df.unstack(level="state").stack(level="side", future_stack=True)

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### Pivoting "Long" to "Wide" Format

First, we create data into a long format.

In [89]:
data = pd.read_csv("../data/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [90]:
periods = pd.PeriodIndex.from_fields(year=data.pop("year"),
                                     quarter=data.pop("quarter"))
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203)

In [91]:
data.index = periods.to_timestamp("D")
data.index.name = 'date'
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [92]:
data.stack()

date        item   
1959-01-01  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-04-01  realgdp     2778.801
            infl           2.340
                         ...    
2009-04-01  infl           3.370
            unemp          9.200
2009-07-01  realgdp    12990.341
            infl           3.560
            unemp          9.600
Length: 609, dtype: float64

Long format: commonly stored in SQL.

In [93]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))
long_data.head()

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


Pivoting makes the data easier to work with. 

In [94]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Pivoting for two value columns gives a hierarchical columns.

In [95]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.855207
1,1959-01-01,infl,0.000,-0.459894
2,1959-01-01,unemp,5.800,-0.661712
3,1959-04-01,realgdp,2778.801,-0.610780
4,1959-04-01,infl,2.340,-0.686321
5,1959-04-01,unemp,5.100,0.989320
6,1959-07-01,realgdp,2775.488,2.003162
7,1959-07-01,infl,2.740,-1.197399
8,1959-07-01,unemp,5.300,0.365757
9,1959-10-01,realgdp,2785.204,-0.387323


In [96]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.459894  0.855207 -0.661712
1959-04-01  2.34  2778.801   5.1 -0.686321 -0.610780  0.989320
1959-07-01  2.74  2775.488   5.3 -1.197399  2.003162  0.365757
1959-10-01  0.27  2785.204   5.6  0.684246 -0.387323 -0.290716
1960-01-01  2.31  2847.699   5.2 -0.889991  1.256532 -1.549396

Pivoting is the same as setting index and unstacking.

In [97]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.459894  0.855207 -0.661712
1959-04-01  2.34  2778.801   5.1 -0.686321 -0.610780  0.989320
1959-07-01  2.74  2775.488   5.3 -1.197399  2.003162  0.365757
1959-10-01  0.27  2785.204   5.6  0.684246 -0.387323 -0.290716
1960-01-01  2.31  2847.699   5.2 -0.889991  1.256532 -1.549396

### Pivoting “Wide” to “Long” Format

Inverse of `pivot` for df is `melt`.

In [98]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [99]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Back to original using `pivot`. Need to use `reset_index` to convert the index back into data. The order of data is different.

In [100]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [101]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


Using `value_vars` to specify columns to melt:

In [102]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


Using `melt` wihtout group identifiers:

In [103]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [104]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
